In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Checking Partial Derivatives with Finite Difference
In addition to using approximations to estimate partial derivatives, you can also use approximations to check your implementations of the partial derivatives for a component.

`Problem` has a method, `check_partials`, that checks partial derivatives
comprehensively for all Components in your model. To do this check, the framework compares the analytic result against a finite difference result. This means that the check_partials function can be quite computationally expensive. So use it to check your work, but don’t leave the call in your production run scripts.

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.check_partials
        :noindex:
```

```{Note}
For components that provide their partials directly (from the `compute_partials` or `linearize` methods, only information about the forward derivatives are shown. For components that are matrix-free, both forward and reverse derivative information is shown.

Implicit components are matrix-free if they define a `apply_linear` method. Explicit components are matrix-free if they define a `compute_jacvec_product` method.
``` 

## Basic Usage
When the difference between the FD derivative and the provided derivative is larger (in either a relative or absolute sense) than 1e-6, that partial derivative will be marked with a '*'.

In [ ]:
import numpy as np

import openmdao.api as om

class MyComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)

        self.add_output('y', 5.5)

        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x1'] + 4.0*inputs['x2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['y', 'x1'] = np.array([4.0])
        J['y', 'x2'] = np.array([40])

prob = om.Problem()

prob.model.add_subsystem('comp', MyComp())

prob.set_solver_print(level=0)

prob.setup(mode='rev')
prob.run_model()

data = prob.check_partials()

In [ ]:
x1_error = data['comp']['y', 'x1']['abs error']

print(x1_error.forward)

In [ ]:
x2_error = data['comp']['y', 'x2']['rel error']

print(x2_error.forward)

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(x1_error.forward, 1.0000000004688445)
assert_near_equal(x2_error.forward, 8.99999999860222)

Turn off standard output and just view the derivatives in the return:

In [ ]:
import numpy as np

import openmdao.api as om

class MyComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)

        self.add_output('y', 5.5)

        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x1'] + 4.0*inputs['x2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['y', 'x1'] = np.array([4.0])
        J['y', 'x2'] = np.array([40])

prob = om.Problem()

prob.model.add_subsystem('comp', MyComp())

prob.set_solver_print(level=0)

prob.setup()
prob.run_model()

In [ ]:
data = prob.check_partials(out_stream=None, compact_print=True)
print(data)

# Show Only Incorrect Printing Option
If you are only concerned with seeing the partials calculations that are incorrect, set `show_only_incorrect` to `True`. This applies to both `compact_print` equal to `True` and `False`.

In [ ]:
import numpy as np
import openmdao.api as om

class MyCompGoodPartials(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)
        self.add_output('y', 5.5)
        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0 * inputs['x1'] + 4.0 * inputs['x2']

    def compute_partials(self, inputs, partials):
        """Correct derivative."""
        J = partials
        J['y', 'x1'] = np.array([3.0])
        J['y', 'x2'] = np.array([4.0])

class MyCompBadPartials(om.ExplicitComponent):
    def setup(self):
        self.add_input('y1', 3.0)
        self.add_input('y2', 5.0)
        self.add_output('z', 5.5)
        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['z'] = 3.0 * inputs['y1'] + 4.0 * inputs['y2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['z', 'y1'] = np.array([33.0])
        J['z', 'y2'] = np.array([40.0])

prob = om.Problem()
prob.model.add_subsystem('good', MyCompGoodPartials())
prob.model.add_subsystem('bad', MyCompBadPartials())
prob.model.connect('good.y', 'bad.y1')

prob.set_solver_print(level=0)
prob.setup()
prob.run_model()

prob.check_partials(compact_print=True, show_only_incorrect=True)
prob.check_partials(compact_print=False, show_only_incorrect=True)

# Running With Multiple FD Step Sizes
If the `step` argument is provided as a list of values instead of a single value, the FD partial derivatives will be evaluated and displayed for each given step size.  This can be useful in certain cases where complex step checks are not possible and the component(s) being checked are expensive to execute.  Supplying multiple FD step sizes in that case will only compute the analytic derivatives once and compare those to each computed FD derivative, which is less expensive than making a separate call to `check_partials` for each FD step size.